<a href="https://colab.research.google.com/github/anskong/ai_ref_from_lessons/blob/main/LGCNS_LangChain_013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 벡터스토어에 내장벡터 저장 (PDF 업로드 → 분할 → 임베딩 → Chroma 저장 및 검색)


In [ ]:
# 1) 필수 패키지 설치
!pip install --quiet --upgrade "langchain-core>=0.3.0" "langchain-community>=0.0.30" "langchain-text-splitters>=0.0.1" "langchain-chroma>=0.0.5" "langchain-openai>=0.0.8" openai pypdf chromadb

In [ ]:
# 2) Colab 파일 업로드
from google.colab import files
import os
from google.colab import userdata

# 3) OPENAI API 키 불러오기
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("📤 PDF 파일을 업로드하세요...")
uploaded = files.upload()
filename = list(uploaded.keys())[0]  # 업로드된 파일 이름

# 4) LangChain 모듈 임포트
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 5) PDF 로딩
loader = PyPDFLoader(filename)
pages = loader.load()

# 6) 텍스트 분할
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
splits = splitter.split_documents(pages)

# 7) 임베딩 모델 정의
embedding = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

# 8) Chroma 벡터스토어 생성 및 저장 (임시 디렉토리 사용)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory="./chroma_db"
)

# 9) 유사도 기반 검색
query = "생성형AI의 최신 동향은?"
docs = vectorstore.similarity_search(query, k=3)

# 10) 검색 결과 출력
for i, doc in enumerate(docs):
    print(f"\n--- 유사 문서 {i+1} ---")
    print(doc.page_content[:300])